Importing the modules

In [1]:
import requests, urllib3, certifi

In [2]:
from bs4 import BeautifulSoup

In [3]:
import pandas as pd

Some outputs are not shown in the file because they print a lot of text and it makes it very difficult to find blocks of code between long walls of text.

Creating PoolManager instance to make requests


In [4]:
http = urllib3.PoolManager(cert_reqs = 'CERT_REQUIRED', ca_certs=certifi.where())

Name of the website

In [5]:
breachlist = "https://oag.ca.gov/privacy/databreach/list"

Make the request - this returns an HTTP response object


In [6]:
# Make the request - this returns an HTTP response object
response = http.request('GET', breachlist)

Create a soup object

In [7]:
# Making the soup object
soup = BeautifulSoup(response.data, "html.parser")

In [ ]:
# How our tree is structured
print(soup.prettify())

Let's print the title of the webpage

In [18]:
soup.title

<title>Search Data Security Breaches | State of California - Department of Justice - Office of the Attorney General</title>

In [10]:
soup.title.string

'Search Data Security Breaches | State of California - Department of Justice - Office of the Attorney General'

In [11]:
soup.a

<a class="element-invisible element-focusable" href="#main-content">Skip to main content</a>

Extract all the hyperlinks on the page.

In [12]:
# Get all the hyperlinks
all_links = soup.find_all("a")

In [ ]:
all_links

In [ ]:
for link in all_links:
    print(link.get("href"))

In [15]:
#Get all tables on the webpage
all_tables = soup.find_all("table")

In [ ]:
all_tables

In [17]:
for table in all_tables:
    print(table.get("class"))

['views-table', 'cols-3', 'table', 'table-hover', 'table-striped']


In [8]:
my_table = soup.find("table", class_ = "views-table cols-3 table table-hover table-striped")

In [ ]:
print(my_table)

In [ ]:
# Find all the rows in the table
soup.findAll("tr")

Now, time to scrape the data present in the table

In [10]:
#Generating lists for each column
A = []
B = []
C = []
for row in soup.findAll("tr"):
    cells = row.findAll("td") #td is a tag for an individual element in a row
    if len(cells) == 3:
        A.append(cells[0].find_all(text=True))
        B.append(cells[1].find_all(text=True))
        C.append(cells[2].find_all(text=True))

In [ ]:
A

In [22]:
A[0][1]

'California Department of Corrections and Rehabilitation'

A is a list of lists, where the first element of each individual list is a line break. We need to extract the second element which is the name of the organization.

In [11]:
org_name = []
for i in range(len(A)):
    org_name.append(A[i][1])        

In [12]:
org_name[0]

'Mindlance, Inc.'

In [ ]:
B, len(B)

B is also a list of lists with a few complications. Lists for entries n/a have just one element with a line break, whitespace, n/a and more whitespace. Entries with dates are lists of multiple elements. The elements include line breaks, dates, commas to separate the dates and a single whitespace.

In [23]:
len(B[0]), len(B[6]), len(B[4])

(3, 5, 3)

In [24]:
B[0][0]

'\n'

We need to be careful of this difference and find a way to extract the dates correctly. So, we will remove all the unnecessary characters to make it easier to get our dates.

In [13]:
# Removing white spaces, line breaks for n/a entries. Joining elements in every of the other lists with a -
B2 = []
for i in range(len(B)):
    if len(B[i]) == 1:
        B2.append(B[i][0].replace(" ", "").replace("\n", ""))
    else:
        B2.append('-'.join(B[i]))

In [ ]:
B2

In [15]:
# Extracting the breach dates in a list
breach_date = []
for i in range(len(B2)):
    if B2[i] != 'n/a':
        breach_date.append(B2[i].replace("-", "").replace("\n", "").strip())
    else:
        breach_date.append(B2[i])

In [ ]:
breach_date

In [ ]:
C

In [30]:
trial = C[0][0].replace(" ", "").replace("\n", "")
trial

'01/05/2018'

Again, we must remove whitespaces from the date reported column.

In [17]:
# Extracting reporting dates - removed whitespaces and line breaks
report_date = []
for i in range(len(C)):
    report_date.append(C[i][0].replace(" ", "").replace("\n", ""))        

In [ ]:
report_date

In [19]:
final_list = [('Organization Name', org_name), ('Date(s) of Breach', breach_date), ('Reported Date', report_date)]

In [20]:
# Making the dataframe
breach = pd.DataFrame.from_items(final_list)

In [22]:
breach.head(15)

,Organization Name,Date(s) of Breach,Reported Date
0,"Mindlance, Inc.",12/28/2017,01/19/2018
1,Rosewood Hotel Group,"05/29/2016, 01/11/2017",01/19/2018
2,Corovan Corporation,09/14/2017,01/18/2018
3,Employer Leasing Company,"09/14/2017, 09/18/2018",01/18/2018
4,American Golf Corporation,"12/12/2017, 12/15/2017",01/18/2018
5,Pension Fund of the Christian Church (“Pension...,11/09/2017,01/16/2018
6,"Guaranteed Rate, Inc.","06/09/2017, 10/02/2017",01/12/2018
7,Pedes Orange County,11/14/2017,01/12/2018
8,"Deli Management, Inc. (d/b/a Jason's Deli, Inc.)",06/08/2017,01/11/2018
9,"talentReef, Inc.",10/16/2017,01/10/2018


In [23]:
updated_org = breach.iloc[0:11, :]

In [24]:
updated_org

,Organization Name,Date(s) of Breach,Reported Date
0,"Mindlance, Inc.",12/28/2017,01/19/2018
1,Rosewood Hotel Group,"05/29/2016, 01/11/2017",01/19/2018
2,Corovan Corporation,09/14/2017,01/18/2018
3,Employer Leasing Company,"09/14/2017, 09/18/2018",01/18/2018
4,American Golf Corporation,"12/12/2017, 12/15/2017",01/18/2018
5,Pension Fund of the Christian Church (“Pension...,11/09/2017,01/16/2018
6,"Guaranteed Rate, Inc.","06/09/2017, 10/02/2017",01/12/2018
7,Pedes Orange County,11/14/2017,01/12/2018
8,"Deli Management, Inc. (d/b/a Jason's Deli, Inc.)",06/08/2017,01/11/2018
9,"talentReef, Inc.",10/16/2017,01/10/2018


In [36]:
breach.shape

(1263, 3)

In [37]:
pd.isnull(breach).sum()

Organization Name    0
Date(s) of Breach    0
Reported Date        0
dtype: int64

Our dataframe is complete. We have the table as shown on the website.  <br>
Each oraganization is a hyperlink to a different page which contains the links to the PDF documents.<br>Now, let us get all those hyperlinks for the organizations and place them in a new column in our dataframe.

In [ ]:
soup.table

In [25]:
import re

All the links are present in a common directory. This makes our job easier.

In [26]:
# Getting all the relevant hyperlinks
all_links = soup.table.find_all("a", href = re.compile("https://oag.ca.gov/ecrime/databreach/reports/"))

In [ ]:
all_links

In [27]:
len(all_links)

1274

In [43]:
type(all_links)

bs4.element.ResultSet

In [28]:
# Exracting the links to the PDF document links
D = []
for link in all_links:
    D.append(link['href'])

In [29]:
D_new = D[0:11]

In [30]:
D_new

['https://oag.ca.gov/ecrime/databreach/reports/sb24-132953',
 'https://oag.ca.gov/ecrime/databreach/reports/sb24-132951',
 'https://oag.ca.gov/ecrime/databreach/reports/sb24-132931',
 'https://oag.ca.gov/ecrime/databreach/reports/sb24-132930',
 'https://oag.ca.gov/ecrime/databreach/reports/sb24-132928',
 'https://oag.ca.gov/ecrime/databreach/reports/sb24-132815',
 'https://oag.ca.gov/ecrime/databreach/reports/sb24-132638',
 'https://oag.ca.gov/ecrime/databreach/reports/sb24-132631',
 'https://oag.ca.gov/ecrime/databreach/reports/sb24-132606',
 'https://oag.ca.gov/ecrime/databreach/reports/sb24-132553',
 'https://oag.ca.gov/ecrime/databreach/reports/sb24-132538']

In [31]:
updated_org['Hyperlinks'] = pd.Series.from_array(D_new)

D:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [33]:
updated_org['Hyperlinks'][0]

'https://oag.ca.gov/ecrime/databreach/reports/sb24-132953'

In [34]:
updated_org.head(15)

,Organization Name,Date(s) of Breach,Reported Date,Hyperlinks
0,"Mindlance, Inc.",12/28/2017,01/19/2018,https://oag.ca.gov/ecrime/databreach/reports/s...
1,Rosewood Hotel Group,"05/29/2016, 01/11/2017",01/19/2018,https://oag.ca.gov/ecrime/databreach/reports/s...
2,Corovan Corporation,09/14/2017,01/18/2018,https://oag.ca.gov/ecrime/databreach/reports/s...
3,Employer Leasing Company,"09/14/2017, 09/18/2018",01/18/2018,https://oag.ca.gov/ecrime/databreach/reports/s...
4,American Golf Corporation,"12/12/2017, 12/15/2017",01/18/2018,https://oag.ca.gov/ecrime/databreach/reports/s...
5,Pension Fund of the Christian Church (“Pension...,11/09/2017,01/16/2018,https://oag.ca.gov/ecrime/databreach/reports/s...
6,"Guaranteed Rate, Inc.","06/09/2017, 10/02/2017",01/12/2018,https://oag.ca.gov/ecrime/databreach/reports/s...
7,Pedes Orange County,11/14/2017,01/12/2018,https://oag.ca.gov/ecrime/databreach/reports/s...
8,"Deli Management, Inc. (d/b/a Jason's Deli, Inc.)",06/08/2017,01/11/2018,https://oag.ca.gov/ecrime/databreach/reports/s...
9,"talentReef, Inc.",10/16/2017,01/10/2018,https://oag.ca.gov/ecrime/databreach/reports/s...


Export the dataframe as a xlsx file

In [35]:
writer = pd.ExcelWriter('Updated_org.xlsx', engine='xlsxwriter')

In [36]:
updated_org.to_excel(writer)

In [37]:
writer.save()

In [6]:
df = pd.read_csv("Updated_org.csv",encoding = 'ISO-8859-1')

In [7]:
df.head()

,Unnamed: 0,Organization Name,Date(s) of Breach,Reported Date,Hyperlinks
0,0,"FlagShip Facility Services, Inc.",12/05/2017,01/05/2018,https://oag.ca.gov/ecrime/databreach/reports/s...
1,1,Rea.deeming Beauty Inc. d/b/a beautyblender,"04/23/2015, 10/26/2017",01/05/2018,https://oag.ca.gov/ecrime/databreach/reports/s...
2,2,LiveGlam Inc.,04/25/2017,01/03/2018,https://oag.ca.gov/ecrime/databreach/reports/s...
3,3,"Forever 21, Inc.",04/03/2017,12/28/2017,https://oag.ca.gov/ecrime/databreach/reports/s...
4,4,"Kaiser Foundation Health Plan, Inc.",10/09/2017,12/28/2017,https://oag.ca.gov/ecrime/databreach/reports/s...


In [25]:
hyperlink1 = df['Hyperlinks'][9]

In [26]:
hyperlink1

'https://oag.ca.gov/ecrime/databreach/reports/sb24-131599'